In [1]:
import tensorflow as tf
import os

In [2]:
# Avoid OOM errors by setting GPU Memory Consumption Growth
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)

In [3]:
tf.config.list_physical_devices('GPU')


[]

In [4]:
import cv2
import imghdr

C:\Users\Administrator\AppData\Local\Temp\ipykernel_18204\4232469594.py:2: DeprecationWarning: 'imghdr' is deprecated and slated for removal in Python 3.13
  import imghdr


In [5]:
data_dir = 'data' 
image_exts = ['jpeg','jpg', 'png']


In [6]:
for image_class in os.listdir(data_dir): 
    for image in os.listdir(os.path.join(data_dir, image_class)):
        image_path = os.path.join(data_dir, image_class, image)
        try: 
            img = cv2.imread(image_path)
            tip = imghdr.what(image_path)
            if tip not in image_exts: 
                print('Image not in ext list {}'.format(image_path))
                os.remove(image_path)
        except Exception as e: 
            print('Issue with image {}'.format(image_path))
            # os.remove(image_path)

Image not in ext list data\Degas\degas_edgar.jpg
Image not in ext list data\Degas\degas_edgar_1.jpg
Image not in ext list data\Degas\degas_edgar_3.jpg
Image not in ext list data\Degas\edgar-degas-update-880.jpg
Image not in ext list data\Lorrain\220px-F0087_Louvre_Gellee_port_au_soleil_couchant-_INV4715_rwk.jpg
Image not in ext list data\Lorrain\Landscape20with20St20Paula20of20Rome20Embarking20at20Ostia20Claude20Lorrain.jpg
Image not in ext list data\Lorrain\lorrain_claude_2.jpg
Image not in ext list data\Monet\Famous-Monet-Paintings.jpg
Image not in ext list data\Monet\monet_claude_525.jpg
Image not in ext list data\Munch\bio_box_munch_edvard.jpg
Image not in ext list data\Munch\munch_edvard.jpg
Image not in ext list data\Munch\munch_edvard_3.jpg
Image not in ext list data\Poussin\banner.jpg
Image not in ext list data\Poussin\neoclassicism_4.jpg
Image not in ext list data\Poussin\poussin_nicolas_6.jpg
Image not in ext list data\VanGogh\van_gogh_vincent.jpg


In [7]:
import numpy as np
from matplotlib import pyplot as plt